In [35]:
import causaldata
import numpy as np
print(dir(causaldata))
df = causaldata.thornton_hiv.load_pandas().data
print(causaldata.nhefs.NOTE)
print(causaldata.nhefs.DESCRLONG)
print(causaldata.nhefs.SOURCE)

['Mroz', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'abortion', 'adult_services', 'auto', 'avocado', 'black_politicians', 'castle', 'close_elections_lmb', 'cps_mixtape', 'credit_cards', 'gapminder', 'google_stock', 'gov_transfers', 'gov_transfers_density', 'greek_data', 'mortgages', 'nhefs', 'nhefs_codebook', 'nhefs_complete', 'nsw_mixtape', 'organ_donations', 'restaurant_inspections', 'ri', 'scorecard', 'snow', 'social_insure', 'texas', 'thornton_hiv', 'titanic', 'training_bias_reduction', 'training_example', 'yule']
::
    Number of observations - 1629
    Number of variables -67
    See the nhefs_codebook data set for variable documentation

nhefs is a cleaned data set of the data used in Causal Inference by Hernán and Robins. nhefs is dataset containing data from the National Health and Nutrition Examination Survey Data I Epidemiologic Follow-up Study (NHEFS). The NHEFS was jointly initiated by th

In [24]:
outcome = 'got'
treatment = 'any'
rest = df.drop([outcome, treatment, 'villnum'], axis=1).columns

In [25]:
#import wooldridge
#df = wooldridge.data('jtrain3')
#df['avg'] = 0.5 * (df.re74+df.re75)
df = df.dropna()
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest]
print(df.shape)
df.head()

(2825, 7)


,villnum,got,distvct,tinc,any,age,hiv2004
0,1.0,1.0,2.718921,2.08032,1.0,22.0,0.0
2,1.0,1.0,2.485713,1.89120,1.0,19.0,0.0
4,1.0,1.0,1.837131,0.09456,1.0,53.0,0.0
5,1.0,1.0,2.217743,0.94560,1.0,50.0,0.0
10,1.0,0.0,2.907790,0.56736,1.0,21.0,0.0


In [26]:
# Simple Comparision of Means
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                    got   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.163
Method:                 Least Squares   F-statistic:                     552.2
Date:                Sun, 04 Dec 2022   Prob (F-statistic):          1.11e-111
Time:                        20:56:59   Log-Likelihood:                -1573.5
No. Observations:                2825   AIC:                             3151.
Df Residuals:                    2823   BIC:                             3163.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.4511      0.019     23.500      0.0

In [27]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                    got   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     132.3
Date:                Sun, 04 Dec 2022   Prob (F-statistic):          2.84e-126
Time:                        20:56:59   Log-Likelihood:                -1528.0
No. Observations:                2825   AIC:                             3068.
Df Residuals:                    2819   BIC:                             3104.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3433      0.024     14.581      0.0

In [30]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 3)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: got
Treatment variable(s): ['any']
Covariates: ['distvct', 'tinc', 'age', 'hiv2004']
Instrument variable(s): None
No. Observations: 2825

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2825 entries, 0 to 4791
Columns: 7 entries, villnum to hiv2004
dtypes: float32(2), float64(5)
memory usage: 154.5 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: got
Treatment variable(s): ['any']
Covariates: ['distvct', 'tinc', 'age', 'hiv2004']
Instrument variable(s): None
No. Observations: 2825

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=3, max_features='sqrt', 

In [31]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,0.343,0.026,13.262,0.0,0.292,0.393
